# Rough example usage

In [16]:
%load_ext autoreload
%autoreload 2

from blackboxbandits import compare, bandits, meta, utils
import pandas as pd

pd.set_option('display.max_rows', None)
pd.options.display.float_format = '{:20,.2f}'.format

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Base optimizer comparison

### Run a new comparison

In [2]:
import pmlb
dataset_names = list(map(lambda x: "clf-"+x.replace("_","-"), pmlb.classification_dataset_names)) \
                + list(map(lambda x: "reg-"+x.replace("_","-"), pmlb.regression_dataset_names))

In [3]:
base_comparison = compare.BaseOptimizerComparison(
    optimizers=["HyperOpt", "OpenTuner-BanditA", "OpenTuner-GA",
                "OpenTuner-GA-DE", "PySOT", "RandomSearch",
                "Scikit-GBRT-Hedge", "Scikit-GP-Hedge", "Scikit-GP-LCB"],
    classifiers=["MLP-adam","lasso"],
    datasets = dataset_names,
    metrics=["mse", "nll"],
    num_calls=20,
    num_repetitions=1,
    db_root = "./saved_results",
    datasets_root="./penn_datasets",
    parallel = True
)

In [ ]:
base_comparison.run()
dbid = base_comparison.get_dbid()
dbid

### View data from a previous comparison

In [ ]:
DBID = "bo_20220228_124924_b7rkpeqg"

In [ ]:
df = compare.BaseOptimizerComparison.get_results_for_dbid(db_root="./saved_results", dbid=DBID)

## Meta optimizer comparison

### Set up meta optimizers

In [ ]:
S = 1
GAMMA = 0.2

In [ ]:
factors = [(a,b) for a in range(1,7) for b in range(1,7) if a*b < 7]

best = {
    f"best_{T}": meta.BestFixedTAlgos(T=T)
    for T in range(2,7)}
fpml_fixedexp = {
    f"fpml_fixedexp_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLFixed,
        T=T, S=S)
    for T in range(2,7)}
fpml_probexp = {
    f"fpml_probexp_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLProb,
        T=T, gamma=GAMMA)
    for T in range(2,7)}
fpml_gr_noexp = {
    f"fpml_gr_noexp_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLWithGR,
        T=T, gamma=0)
    for T in range(2,7)}
fpml_gr_probexp = {
    f"fpml_gr_probexp_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.FPMLWithGR,
        T=T, gamma=GAMMA)
    for T in range(2,7)}
streeter_fpml = {
    f"streeter_fpml_{a}x{b}": meta.BanditMetaOptimizer(
        bandit_type=bandits.StreeterFPML,
        T=a*b, T_1=a, T_2=b, gamma=GAMMA, gr=True)
    for a,b in factors}
streeter_exp3 = {
    f"streeter_exp3_{T}": meta.BanditMetaOptimizer(
        bandit_type=bandits.Streeter,
        T=T)
    for T in range(2,7)}

meta_optimizers = {**best, **fpml_fixedexp, **fpml_probexp, **fpml_gr_noexp,
                   **fpml_gr_probexp, **streeter_fpml, **streeter_exp3}

### Setup and run meta comparison

In [18]:
meta_comparison = compare.MetaOptimizerComparison \
                         .from_precomputed_base_comparison(
    dbid=DBID,
    meta_optimizers=meta_optimizers,
    db_root = "./data",
    parallel_meta = False,
    num_meta_repetitions = 1
)

NameError: name 'meta_optimizers' is not defined

In [35]:
meta_comparison.run_meta_comparison()
results = meta_comparison.summary()
results

,visible_score,generalization_score
optimizer,,
HyperOpt_0.2.7_a376313,0.33,0.28
OpenTuner-BanditA_0.8.7_a376313,0.34,0.36
OpenTuner-GA-DE_0.8.7_a376313,0.38,0.28
OpenTuner-GA_0.8.7_a376313,0.25,0.22
PySOT_0.3.3_a376313,0.37,0.30
RandomSearch_0.0.8_a376313,0.19,0.28
Scikit-GBRT-Hedge_0.9.0_a376313,0.49,0.40
Scikit-GP-Hedge_0.9.0_a376313,0.53,0.49
Scikit-GP-LCB_0.9.0_a376313,0.58,0.35


In [55]:
utils.visualise_by_budget(results)

interactive(children=(IntSlider(value=1, description='T', max=7, min=1), Output()), _dom_classes=('widget-inte…